# Coding your own RNN

Using this pre-filled notebook, we will code our own RNN for sentence classification. For now, we'll keep using IMDB, as the goal of this part is to understand how an RNN works.

Unlike our previous lab, we will also learn the embedding layer. Which means we need to deal with vocabulary by ourselves.

In [1]:
from functools import partial
from typing import Callable, Dict, Generator, List, Tuple

from datasets import load_dataset
import numpy as np
from sklearn.utils import shuffle
import torch
from torch import nn
from torchtext.vocab import vocab, Vocab
from torchtext.data.utils import get_tokenizer

from tqdm.auto import tqdm

## Dataset
We load the dataset and split the training set in a stratified train/validation set.

In [2]:
dataset = load_dataset("imdb")
train_dataset = dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]
train_df.shape, valid_df.shape, test_df.shape

Found cached dataset imdb (/home/ethan/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/ethan/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-5f37fd0866e4f89f.arrow and /home/ethan/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-dd5732a0e6ac784c.arrow


((20000, 2), (5000, 2), (25000, 2))

## Vocabulary (1 point)

**\[1 point\]** Build your own vocabulary. The [example provided in torchtext documentation](https://pytorch.org/text/stable/vocab.html#id1) might be of help.
* Don't forge to setup the `min_freq` parameter to not include unfrequent noise.
* You will need a tokenizer. Reuse the `basic_english` one from the our previous lab.
* For an RNN we need two special tokens: `<unk>`, for unknown words, and `<pad>` for padding.

In [3]:
def preprocess(base_text):
    """
    Preprocess the text before classification
    Args:
        base_text: the string to preprocess
    Return:
        The preprocessed text
    """
    return base_text.replace("<br />",' ')

def create_vocab(texts, frequencies = {}):
    for text in texts:
        for word in preprocess(text).split(" "):
            if word not in frequencies:
                frequencies[word] = 1
                continue
            frequencies[word] += 1
    return dict(sorted(frequencies.items(), key=lambda x: x[1], reverse=True))
frequencies = create_vocab(train_df["text"])

In [4]:
tokenizer = get_tokenizer("basic_english", language="en")
vocabulary = vocab(frequencies, min_freq = 100, specials=["<unk>", "<pad>"])
vocabulary.set_default_index(0)

## Vectorize and batch the input (3 points)

As seen in class, our model should take one-hot encoded vectors corresponding to the each token vocabulary id. However, computing a vector x matrix multiplication for every input is unnecessarily costly. Multiplying a one-hot vector with a matrix is the equivalent of taking one row of the matrix. In pyTorch, we provide ids for each token which will be used as input to an `nn.Embedding` layer. The id is simply the row in the embedding matrix.

**\[1 point\]** Fill the `vectorize_text` function returning a 1D torch tensor of `torch.long` for each input text.

In [5]:
def vectorize_text(
    text: str, vocabulary: Vocab, tokenizer: Callable[[str], List[str]]
) -> torch.Tensor:
    """
    Generate a tensor of vocabluary IDs for a given text.
    Args:
        text: the input text.
        vocabulary: a Vocab objects.
        tokenizer: a text tokenizer.
    Returns:
        A tensor of IDs (torch.long).
    """
    tokens = tokenizer(preprocess(text))
    tokens_ids = vocabulary.lookup_indices(tokens)
    return torch.tensor(tokens_ids, dtype=torch.long)

In [6]:
text_pipeline = partial(vectorize_text, vocabulary=vocabulary, tokenizer=tokenizer)

Check the function is working correctly, especially it should return the right special id for unknown words.

In [7]:
text_pipeline("Some text I am thinking about... ragafqfa")

tensor([ 47,   0, 143, 229, 556,  42, 425, 425, 425,   0])

Result:
* tensor([ 47,   0, 143, 229, 556,  42, 425, 425, 425,   0])

In [8]:
X_train = [text_pipeline(text) for text in tqdm(train_df["text"])]
y_train = train_df["label"]
X_valid = [text_pipeline(text) for text in tqdm(valid_df["text"])]
y_valid = valid_df["label"]
X_test = [text_pipeline(text) for text in tqdm(test_df["text"])]
y_test = test_df["label"]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/25000 [00:00<?, ?it/s]

To speed up the training process, we turn the inputs into batches, as we did last time. For batches to work, every line must have the same lengths. Last time, it was implicit as only a vector (the average of all embeddings) was provided. This time, every line has the length of a different review.

To go around this problem, we use padding. So every line within a batch is padded to the length of its longest element.

* **\[1 point\]** Fill the data generator function.
* **\[1 point\]** On which side should you pad and why?

In [9]:
def data_generator(
    X: List[torch.tensor], y: List[int], pad_id: int, batch_size: int = 32
) -> Generator[Tuple[torch.Tensor, torch.Tensor], None, None]:
    """
    Yield batches from given input data and labels.
    Args:
        X: a list of tensor (input features).
        y: the corresponding labels.
        batch_size: the size of every batch [32].
    Returns:
        A tuple of tensors (features, labels).
    """
    X, y = shuffle(X, y)
    n_batches = len(X) // batch_size
    for i in range(n_batches):
        batch_X = X[i * batch_size : (i + 1) * batch_size]
        batch_y = y[i * batch_size : (i + 1) * batch_size]
        batch_X = nn.utils.rnn.pad_sequence(batch_X, padding_value=pad_id)
        yield batch_X, torch.tensor(batch_y, dtype=torch.long)

In [10]:
pad_token = "<pad>"
train_gen = lambda: data_generator(X_train, y_train, vocabulary[pad_token])
valid_gen = lambda: data_generator(X_valid, y_valid, vocabulary[pad_token])
test_gen = lambda: data_generator(X_test, y_test, vocabulary[pad_token])

## Classifier (3 points)

**\[3 points\]** Code your own RNN. Fill the `RNN` class correctly. Remember an RNN has 3 matrices and an embedding layer (see course slide 61).
* The embedding layer turns a one-hot vectors into dense vectors.
* The first matrix (W) connects the embedding to the hidden layer.
  * `embedding_size -> hidden_size`
* The second matrix (U) connect the previous hidden layer to the current one.
  * `hidden_size -> hidden_size`
* These to vectors are added and go through an activation function (e.g. $h_t = tanh(Wx_i+Uh_{t-1})$).
* The last matrix (V) connects the hidden layer to the hidden layer to the output.
  * `hidden_size -> 1`
* Donc forget to add an `init_hidden` function which initialize the first hidden layer to 0.

In [11]:
from typing import Iterator


from torch.nn.parameter import Parameter


class RNN(nn.Module):
    """A simple RNN module with word embeddings.
    """
    def __init__(self, vocab_size: int, embed_size: int, hidden_size, device):
        """
        Args:
            vocab_size: vocabulary size.
            embed_size: embedding dimensions.
            hidden_size: hidden layer size.
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.device = device


        # The word embedding layer.
        self.embed = nn.Embedding(self.vocab_size, self.embed_size, device = self.device)
        self.U = nn.Linear(self.hidden_size, self.hidden_size, device=self.device)
        self.V = nn.Linear(self.hidden_size, 1, device=self.device)
        self.W = nn.Linear(self.embed_size, self.hidden_size, device=self.device)

    def parameters(self) -> Iterator[Parameter]:
        yield self.W.weight
        yield self.U.weight
        yield self.V.weight

    def h(self, t, X):
        if t == 0:
            return torch.zeros(self.hidden_size, X.size(2)).to(self.device)
        Uh = self.U(self.h(t-1, X).transpose(0, 1))
        Uh = Uh.expand(X.size(0), Uh.size(0), Uh.size(1))
        Wx = self.W(X)
        return torch.tanh(Uh + Wx).to(self.device)

    def forward(self, X: torch.Tensor):

        embed_X = self.embed(X)
        h = self.h(1, embed_X)
        y = self.V(h.transpose(0, 1))
        return y.reshape(y.size(0), y.size(1))      


## Training (2 points)

Training is a bit different than usual. We will need to sequentially (but in "batch parallel") go through an input, keeping track of the hidden layer, and use the last output as prediction.

**\[2 point\]** Code the training loop.
* Note that for each batch, you need to loop through the whole input and use the output of the last token as input to your criterion.
* Keep the best model evaluated on the validation set.
* Plot the training and validation losses.
* Training will take some time (~30 min on a T4 GPU). Make sure your results appear in the notebook.

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [13]:
def train(
    model: nn.Module,
    criterion: Callable,
    optimizier: torch.optim.Optimizer,
    n_epochs: int,
    train_gen: Callable,
    valid_gen: Callable,
):
    best_model = model
    best_loss = np.inf
    i=0
    for _ in tqdm(range(n_epochs), desc="Epochs"):

        for X, y in train_gen():
            X = X.to(device)
            y = y.to(device)
            optimizier.zero_grad()
            y_pred = model(X)
            y = y.reshape(y.size(0), 1).expand(y.size(0), y_pred.size(1))
            loss = criterion(y_pred, y.float())
            loss.backward()
            optimizier.step()

        if i % 20 == 0:
            print(f"Training loss: {loss.item()}")
            
        for X, y in valid_gen():
            X = X.to(device)
            y = y.to(device)
            with torch.no_grad():
                y_pred = model(X)
                y = y.reshape(y.size(0), 1).expand(y.size(0), y_pred.size(1))
                loss = criterion(y_pred, y.float())
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_model = model
                    
        if i % 20 == 0:
            print(f"Best Validation loss: {best_loss}")

        i+=1

    return best_model


In [14]:
n_embedding = 32
n_hidden = 64
model = RNN(len(vocabulary.get_itos()), n_embedding, n_hidden, device).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [15]:

best_model = train(model, criterion, optimizer, 1000, train_gen, valid_gen)

Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Training loss: 0.6944425702095032
Best Validation loss: 0.6890242695808411
Training loss: 0.6899073123931885
Best Validation loss: 0.6747045516967773
Training loss: 0.6898512840270996
Best Validation loss: 0.6747045516967773
Training loss: 0.6929174661636353
Best Validation loss: 0.6747045516967773
Training loss: 0.7056082487106323
Best Validation loss: 0.6720477938652039
Training loss: 0.6918027400970459
Best Validation loss: 0.6655289530754089
Training loss: 0.6927348375320435
Best Validation loss: 0.6655289530754089
Training loss: 0.6908631324768066
Best Validation loss: 0.6655289530754089
Training loss: 0.6956551671028137
Best Validation loss: 0.6492127180099487
Training loss: 0.6875395178794861
Best Validation loss: 0.6492127180099487
Training loss: 0.6935694813728333
Best Validation loss: 0.6492127180099487
Training loss: 0.698187530040741
Best Validation loss: 0.6492127180099487
Training loss: 0.693101167678833
Best Validation loss: 0.6492127180099487
Training loss: 0.6905004382

Last result:
* Training loss: 0.6944473385810852
* Best Validation loss: 0.642731785774231

## Evaluation (1 point)

* **\[1 point\]** Compute the accuracy for all 3 splits.

In [28]:
def compute_accurracy(X, y):
    X = nn.utils.rnn.pad_sequence(X, padding_value=vocabulary[pad_token])
    X = X.to(device)
    with torch.no_grad():
        y_pred = best_model(X)
        y_pred = y_pred.cpu().detach().numpy()
        y_pred = np.where(y_pred > 0.5, 1, 0)
        return (y_pred == y).sum().item() / len(y)

In [29]:
accuracy_train = compute_accurracy(X_train, y_train)
accuracy_valid = compute_accurracy(X_valid, y_valid)
accuracy_test = compute_accurracy(X_test, y_test)
print(f"Training accuracy: {accuracy_train}")
print(f"Validation accuracy: {accuracy_valid}")
print(f"Test accuracy: {accuracy_test}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.56 GiB (GPU 0; 5.79 GiB total capacity; 857.74 MiB already allocated; 4.20 GiB free; 944.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Results:
* Could not compute accuracy because of Error CUDA out of memory.